# Draw genetic constructs in different states
Uses dnaplotlib:
Der B.S., Glassey E., Bartley B.A., Enghuus C., Goodman D.B., Gordon D.B., Voigt C.A., Gorochowski T.E., "DNAplotlib: programmable visualization of genetic designs and associated data", ACS Synthetic Biology, 2016. (DOI: 10.1021/acssynbio.6b00252)

In [1]:
import dnaplotlib as dpl
import matplotlib.pyplot as plt
import colorcet as cc

# Create the DNAplotlib renderer
dr = dpl.DNARenderer()

In [2]:
class DNA_Part(object):
    """Gene circuit part class."""
    
    def __init__(self, label="", parttype="", state="on", fwd=True, color="", label_y_offset=None,):
        """
        Instantiate part object.
        
        Use LaTex for subscripts, e.g. $\mathrm{P_{R}}$
        
        Part type is "cds", "promoter", "terminator", or "rbs".
        Color is "black", "white", "blue", "orange", "green", "red", "purple", "brown", "pink", "gray".
        """
        # Globals
        colors = cc.glasbey_category10
        color_map = {
            "black": (0.00, 0.00, 0.00),
            "white": (1.00, 1.00, 1.00),
            "blue": colors[0],
            "orange": colors[1],
            "green": colors[2],
            "red": colors[3],
            "purple": colors[4],
            "brown": colors[5],
            "pink": colors[6],
            "gray": colors[7], 
        }
        
        # Set part characteristics
        if parttype.lower() in ("cds", "rbs"):
            self.parttype = parttype.upper()
        else:
            self.parttype = parttype[0].upper() + parttype[1:].lower()
        self.fwd = fwd
        self.state = state
        
        # Opts that are constant
        self.linewidth = 1.0
        self.edge_color = color_map["black"]
        
        # Opts that are user-defined
        self.label = label
        if color:
            self.color = color_map[color]
        elif self.parttype.lower() in ("rbs", "terminator"):
            self.color = color_map["black"]
        if label_y_offset:
            self.label_y_offset = label_y_offset
        
        # Opts that depend on part type
        if not label_y_offset:
            if self.parttype.lower() == "cds":
                self.label_y_offset = -1
            elif self.fwd:
                self.label_y_offset = -8
            else:
                self.label_y_offset = 8
                
        if self.parttype.lower() == "cds":
            self.label_x_offset = -3
            self.label_style = "italic"
            self.label_color = color_map["white"]
        else:
            self.label_x_offset = 0
            self.label_style = None
            self.label_color = color_map["black"]
            
        if self.parttype.lower() == "terminator":
            self.start_pad = -1
        else:
            self.start_pad = None
        
    # Construct dictionary defining part for dnaplotlib
    def on(self):
        """Returns dictionary that represents part for dnaplotlib"""
        part = {
            "type": self.parttype,
            "fwd": self.fwd,
            "opts": {
                "label": self.label,
                "linewidth": self.linewidth,
                "color": self.color,
                "edge_color": self.edge_color,
                "label_style": self.label_style,
                "label_color": self.label_color,
                "label_x_offset": self.label_x_offset,
                "label_y_offset": self.label_y_offset,
            }
        }
        
        if self.start_pad:
            part["opts"]["start_pad"] = self.start_pad
        
        return part
    
    # Construct dictionary defining part when OFF for dnaplotlib
    def off(self):
        """Returns dictionary that represents part when OFF for dnaplotlib"""
        
        def lighten(color):
            """Lighten OFF components"""
            fac = 0.7
            off_color = [0, 0, 0]
            for i, _ in enumerate(color):
                off_color[i] = color[i] + (fac * (1 - color[i]))
            return tuple(off_color)
        
        part = {
            "type": self.parttype,
            "fwd": self.fwd,
            "opts": {
                "label": self.label,
                "linewidth": self.linewidth,
                "color": lighten(self.color),
                "edge_color": self.edge_color,
                "label_style": self.label_style,
                "label_color": self.label_color,
                "label_x_offset": self.label_x_offset,
                "label_y_offset": self.label_y_offset,
            }
        }
        
        if self.start_pad:
            part["opts"]["start_pad"] = self.start_pad
        
        return part

In [3]:
class Reg_Arc(object):
    """Gene circuit regulatory arc class."""
    
    def __init__(self, arctype="Repression", from_part=None, to_part=None):
        """
        Instantiate regulatory arc object.
        
        Arc type is "Activation" or "Repression"
        
        """

        # Set part characteristics
        if arctype.lower() == "activation":
            self.arctype = "Activation"
        else:
            self.arctype = "Repression"
        
        # Opts that are constant
        self.linewidth = 1.0
        
        # Opts that are user-defined
        self.from_part = from_part
        self.to_part = to_part
        
        if from_part:
            self.color = from_part['opts']['color']
        
    # Construct dictionary defining part for dnaplotlib
    def on(self):
        """Returns dictionary that represents part for dnaplotlib"""
        
        return {
            "type": self.arctype,
            "from_part": self.from_part,
            "to_part": self.to_part,
            "opts": {
                "linewidth": self.linewidth,
                "color": self.color,
            }
        }

In [4]:
def render(design_low, design_high, reg_low, reg_high, savename="test.svg"):
    """
    Render genetic circuits in low temperature and high temperature states.
    
    design_low, design_high are lists containing the DNA_Part parts in the order in which they appear
    reg_low, reg_high are lists containing the Reg_Arc regulatory arcs
    """
    
    # Set up the axes for the genetic constructs

    axis_high = plt.axes([0, float(0.75), 1, 0.2])
    axis_low = plt.axes([0, float(0.75)-float(0.2), 1, 0.2])
    
    # Render the DNA to axis
    reg_renderers = dr.std_reg_renderers()
    part_renderers = dr.SBOL_part_renderers()
    
    start, end = dr.renderDNA(axis_low, design_low, dr.SBOL_part_renderers(), reg_low, dr.std_reg_renderers())
    axis_low.set_xlim([start, end])
    axis_low.set_ylim([-27, 27])
    axis_low.set_aspect('equal')
    axis_low.set_xticks([])
    axis_low.set_yticks([])
    axis_low.axis('off')
    
    start, end = dr.renderDNA(axis_high, design_high, dr.SBOL_part_renderers(), reg_high, dr.std_reg_renderers())
    axis_high.set_xlim([start, end])
    axis_high.set_ylim([-27, 27])
    axis_high.set_aspect('equal')
    axis_high.set_xticks([])
    axis_high.set_yticks([])
    axis_high.axis('off')
    
    plt.savefig(savename, bbox_inches='tight',pad_inches = 0, transparent=True)

In [5]:
pR = DNA_Part(label='$\mathrm{P_{R}}$', color="blue", parttype="promoter")
pL = DNA_Part(label='$\mathrm{P_{L}}$', color="blue", parttype="promoter")
pLacI = DNA_Part(label='$\mathrm{P_{LacI}}$', color="black", parttype="promoter")
tci = DNA_Part(label='tcI', color="blue", parttype="cds")
gfp = DNA_Part(label='gfp', color="green", parttype="cds")
T7 = DNA_Part(label='T7', parttype="terminator")

In [6]:
pR_OFF = pR.off()
pL_OFF = pL.off()
pLacI_OFF = pLacI.off()
tci_OFF = tci.off()
gfp_OFF = gfp.off()
T7_OFF = T7.off()

pR = pR.on()
pL = pL.on()
pLacI = pLacI.on()
tci = tci.on()
gfp = gfp.on()
T7 = T7.on()

tci_reg = Reg_Arc(arctype='repression', from_part=tci_OFF, to_part=pR_OFF)
tci_reg2 = Reg_Arc(arctype='repression', from_part=tci_OFF, to_part=pL_OFF)

tci_reg = tci_reg.on()
tci_reg2 = tci_reg2.on()

design_low = [pR_OFF, pL_OFF, gfp_OFF, pLacI, tci_OFF, T7]
design_high = [pR, pL, gfp, pLacI, tci, T7]

reg_low = [tci_reg, tci_reg2]
reg_high = []

In [7]:
render(design_low, design_high, reg_low, reg_high, savename="test.svg")

In [8]:
%load_ext watermark
%watermark -m -v -p matplotlib,colorcet,jupyterlab

CPython 3.8.3
IPython 7.16.1

matplotlib 3.2.2
colorcet 2.0.2
jupyterlab 2.1.5

compiler   : MSC v.1916 64 bit (AMD64)
system     : Windows
release    : 10
machine    : AMD64
processor  : Intel64 Family 6 Model 142 Stepping 10, GenuineIntel
CPU cores  : 8
interpreter: 64bit
